# Clase del 6/10/2020 - Video 9
- Material: Slicer4minute.mrb
- Manual para crear modulos "HelloPython"

# 1: Adquisicion / Manipulacion / Generacion de datos


In [ ]:
# Adquisicion de datos
escena =  slicer.mrmlScene                               # Adquisicion de la escena
volumen = escena.GetNodeByID('vtkMRMLScalarVolumeNode1') # Adquisicion del volumen de la resonancia "grayscale"
imagen = volumen.GetImageData()                          # Datos que constituyen la imagen

# Generacion de datos
volumen_vacio = slicer.vtkMRMLScalarVolumeNode()         # Se crea un volumen vacio
arreglo_numpy = [...]                                    # Se genera un arreglo de numpy
arreglo_numy_vtk = vtk.vtkImageData(arreglo_numpy)       # El arreglo se asigna a un objeto vtkImageData
volumen_vacio.SetAndObserveImageData(imagen)             # el objeto vtkImageData se le asigna al volumen creaado

# Manipulacion de propiedades
modelo_piel = escena.GetNodeByID('vtkMRMLModelNode12')   #ID del modelo de la piel
Luego buscar algun metodo que permita modificar transparencia

# 2: Manipulacion de Imagen / Matriz de datos / Matricez de Transformacion

In [ ]:
# Imagen sin datos creada
imagen_vacia = vtk.vtkImageData()             # Se crea la imagen
imagen_vacia.SetDimensions([10,10,2])         # Se asigna un tamaño
imagen_vacia.SetOrigin([0,0,0])               # Se asigna el origen de la imagen
imagen_vacia.SetSpacing([0.78,0.78,1.5])      # Se asigna el espaciamiento entre puntos de la matriz / Tamaño de Voxeles
imagen_vacia.AllocateScalars(vtk.VTK_SHORT,1) # Se determina el Tipo de datos que almacenara

# Dos formas para asignarlas a un volumen vacio creado
volumen_vacio = slicer.vtkMRMLScalarVolumeNode()    # Se crea un volumen vacio
volumen_vacio.SetAndObserveImageData(imagen_vacia)  # Se le asigna el objeto vtkImageData() creado
volumen_vacio.SetName('Prueba')                     # Se le asigna el nombre de prueba
slicer.mrmlScene.AddNode(volumen_vacio)             # Se añade a la Escena como un **Nodo**

# Otra forma de añadir los datos a la escena de 3D Slicer:
escena = slicer.mrmlScene                           # Instanciar la escena previamente y usarla
escena.mrmlScene.AddNode(volumen_vacio)             # Se añade a la Escena como un **Nodo**

# Extraccion de propiedades de la imagen
datos = imagen_vacia.GetPointData().GetScalars()

In [ ]:
# Adquisicion de la imagen de una escena, instanciados en un objeto vtkMRMLScalarVolumeNode() del tipo vtk.ImageData()
escena =  slicer.mrmlScene                               # Adquisicion de la escena
volumen = escena.GetNodeByID('vtkMRMLScalarVolumeNode1') # Adquisicion del volumen de la resonancia "grayscale"
imagen = volumen.GetImageData()                          # Datos que constituyen la imagen (objeto vtk.ImageData() )

# Adquisicion de los datos de la imagen, en forma de ndarray
datos_escalares_de_la_imagen = imagen.GetPointData().GetScalars()
matriz_ndarray = vtk.util.numpy_support.vtk_to_numpy(datos_escalares_de_la_imagen)
print(type(matriz_ndarray))
print(matriz_ndarray.shape)                              # El ndarray tiene un tamaño extraño para manipular

# Es posible invertir el proceso (ndarray --a--> vtk)
matriz  = vtk.util.numpy_support.numpy_to_vtk(matriz_ndarray)

# Recuperacion de las matricez de transformacion del volumen "grayscale"
# Creemos las matrices vacias de ambos espacios
ras2ijk = vtk.vtkMatrix4x4()       # Matriz para ir al espacio (i,j,k)
ijk2ras = vtk.vtkMatrix4x4()       # Matriz para ir al espacia (RAS)
# Al aplicar estos comandos, las matricez vacias se llenan con la matriz de transformacion de cada espacio
volumen.GetRASToIJKMatrix(ras2ijk)
volumen.GetIJKToRASMatrix(ijk2ras)

# 3 vtk.reslice para remuestrear una imagen y acomodarla

In [ ]:
# 1 Datos del volumen de partida
volumen = slicer.mrmlScene.GetNodeByID('vtkMRMLScalarVolumeNode1') # Recuperamos el volumen "grayscale"
imagen = volumen.GetImageData()                                    # Recuperamos la imagen del volumen 

ras2ijk = vtk.vtkMatrix4x4()                   # Determinemos las matrices de transformacion 
ijk2ras = vtk.vtkMatrix4x4()                   # del volumen "grayscale" recuperado
volumen.GetRASToIJKMatrix(ras2ijk)
volumen.GetIJKToRASMatrix(ijk2ras)

[oX,oY,oZ] = volumen.GetOrigin()               # Recuperemos el origen del volumen para poder  
                                               # realizar la transformacion adecuadamente (min 37:44)

# 2 Generemos la matriz de transformacion
transformacion_1 = vtk.vtkTransform()          # Matriz base de transformacion (Matriz Identidad)

transformacion_1.Translate(*[-oX,-oY,-oZ])     # Se translada todo en una magnitud = -(origen de la imagen)

transformacion_1.RotateX(0.5)                  # Rotacion en X
transformacion_1.RotateY(-0.5)                 # Rotacion en Y

transformacion_1.Translate(*[oX,oY,oZ])         # Regresamos a la posicion donde estaba la imagen transladando
                                               # todo en una magnitud = (origen de la imagen)

# 3 Instanciemos el objeto de clase vtkImageReslice()
reslice = vtk.vtkImageReslice()               # Se instancia el objeto de la clase Reslice para reuestrear
reslice.SetInputData(imagen)                  # Metodo para asociar datos de la imagen al filtro
reslice.SetResliceTransform(transformacion_1) # Se aplica la transformacion
reslice.InterpolateOn()                       # Calculo mediante interpolacion para evitar que falten puntos
reslice.SetBackgroundLevel(-1024)             # Se agrega un nivel de fondo para visualizar mejor

resultado_del_reslice = reslice.GetOutput()   # Se solicita la salida del filtro

# 4 Se genera el volumen resultante
volumen_resultante = slicer.vtkMRMLScalarVolumeNode()             # Se crea un volumen contenedor
volumen_resultante.SetAndObserveImageData(resultado_del_reslice)  # Se le establece el resultado del Reslice

reslice.Update()                               # Es necesario actualizar el filtro despues de cada transformacion aplicada

# 5 Asignacion de matricez de transformacion
volumen_resultante.SetRASToIJKMatrix(ras2ijk)  # Al volumen resultante se le asignan las mismas
volumen_resultante.SetIJKToRASMatrix(ijk2ras)  # matricez de transformacion del mismo volumen de partida

# 6 Asignacion del origen de coordenadas
volumen_resultante.SetOrigin(oX,oY,oZ)         # Se asigna al volumen resultante el mismo origen del volumen de partida

# 7 Adicion de transformaciones del volumen original al volumen resultante (min 47:30)
try:
    volumen_resultante.SetAndObserveTransformNodeID((volumen.GetParentTransformNode()).GetID())
except:
    pass

# 8 Finalmente se añade el volumen resultante a la Escena como un "Nodo"
slicer.mrmlScene.AddNode(volumen_resultante)

# 4 Fiducials para estudios cuantitativos

In [ ]:
markup = slicer.mrmlScene.GetNodeByID("vtkMRMLMarkupsFiducialNode1") # Recuperamos 1 fiducial
ras = [0,0,0]                                # Aqui almacenaremos las ubicaciones en RAS de los fiducials creados
numFids = markup.GetNumberOfFiducials()      # Permite saber cuantos fiducials se han puesto

for i in range(numFids):                     # Recorremos los fiducials creados
    markup.GetNthFiducialPosition(i, ras)    # y almacenamos sus posicions en el vector ras
    print str(ras)

# 5 Ejemplo de Graficas

In [ ]:
import numpy as np;

numero_frames = 100;                  # Simula la cantidad de volumenes de un estudio 4D
dato = np.zeros((100,2));             # Matriz de ceros para rellenar

for i in range(numero_frames):
    dato[i,0] = i                     # Datos del eje X
    dato[i,1] = i**2;                 # Datos del eje Y
    
    chartNode = slicer.util.plot(dato, xColumnIndex=0, columnNames=['X', 'X^2'])
    chartNode.SetXAxisTitle('X')
    chartNode.SetYAxisTitle('Y')
    chartNode.LegendVisibilityOff()
    chartNode.SetTitle('Grafica de Intensidad')

# Parte 6: Identificar parametros de un modulo y ejecutarlo

In [ ]:
# Este script permite identificar los parametros que presenta algun modulo
cliModule = slicer.modules.gradientanisotropicdiffusion # Probemos con el gradientanisotropicdiffusion
n = cliModule.cliModuleLogic().CreateNode()
for groupIndex in range(n.GetNumberOfParameterGroups()):
    for parameterIndex in range(n.GetNumberOfParametersInGroup(groupIndex)):  
        print('Parameter ({0}/{1}): {2} ({3})'.format(groupIndex, parameterIndex, n.GetParameterName(groupIndex, parameterIndex), n.GetParameterLabel(groupIndex, parameterIndex)))

# Se especifican los volumenes que interactuan
volumen_entrada = slicer.mrmlScene.GetNodeByID('vtkMRMLScalarVolumeNode1')
volumen_salida = slicer.vtkMRMLScalarVolumeNode()
slicer.mrmlScene.AddNode(volumen_salida)

# Parametros para la operacion de registro
parameters = {}
parameters['conductance'] = 1.0 
parameters['numberOfIterations'] = 10
parameters['timeStep'] = 0.05
parameters['inputVolume'] = volumen_entrada.GetID()
parameters['outputVolume'] = volumen_salida.GetID()

# Se invoca el modulo de interes y se ejecuta
cliModule = slicer.modules.gradientanisotropicdiffusion
cliNode = slicer.cli.run(cliModule,None,parameters,wait_for_completion=True)